Input: A set of topics in free-text, creates various query representations based on identified, drugs, dosages, problems, treatments, tests

# Intall Libraries, Run utlity functions, and import the queries (raw topics)

In [1]:
!pip install regex
!pip install transformers
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
!pip install medspacy
!pip install spacy==3.0.6
!pip install git+https://github.com/explosion/spacy-transformers
!pip install --upgrade ir_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 980.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.2 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.4 MB/s eta 0:00:00
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578479 sha256=b17bd19c82df7fa8b666eaba99aec0b1515c8964cd914834230f15c84525b116
  Stored in dire

In [2]:
import regex as re
import pandas as pd
from transformers import pipeline
import scispacy
import spacy
import string
import medspacy
from bs4 import BeautifulSoup
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import warnings
warnings.filterwarnings("ignore")
import ir_datasets

##Model for NER recognition
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
model = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner")

##Model for Negated content identification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
tokenizer2 = AutoTokenizer.from_pretrained("bvanaken/clinical-assertion-negation-bert")
model2 = AutoModelForSequenceClassification.from_pretrained("bvanaken/clinical-assertion-negation-bert")

## Load the queries

In [ ]:
# Queries
dataset = ir_datasets.load("clinicaltrials/2021/trec-ct-2022")
queries = pd.DataFrame(dataset.queries_iter())
del dataset
display(queries)

## Utility Functions

In [4]:
"""Gets a topic returns sentences/segments splitted using spacy model
"""
nlp_sentence= spacy.load("en_core_sci_sm")
def split_sent(topic,nlp_sentence):
  doc = nlp_sentence(topic)
  sentences = list(doc.sents)
  return sentences

"""Get a sentence identifies/returns NERs -- MODEL samrawal/bert-base-uncased_clinical-ner [Problem, treatment, test]
"""
def get_entities(sentence):
  nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
  topic_ent = nlp(str(sentence))
  return topic_ent

"""Get a sentence and all its N entities, marked as problem; outputs N senteneces in which the entily is marked ['entity']. This format is required by the transformer based model.
"""
def modify_segment_negation(sentence, topic_ent,extracted_entity: list):
  modified_sentences = []
  inserttxt = "[entity]"
  for ent in topic_ent:
    if ent['entity_group'] == 'problem' and extracted_entity[0] == 'problem':
      modified_sent = sentence[:int(ent['start'])] + inserttxt + ' ' + sentence[int(ent['start']):int(ent['end'])] + ' ' + inserttxt + sentence[int(ent['end']):]
      modified_sentences.append(modified_sent)
    elif ent['entity_group'] == 'treatment'and extracted_entity[1] == 'treatment':
      modified_sent = sentence[:int(ent['start'])] + inserttxt + ' ' + sentence[int(ent['start']):int(ent['end'])] + ' ' + inserttxt + sentence[int(ent['end']):]
      modified_sentences.append(modified_sent)
    elif ent['entity_group'] == 'test' and extracted_entity[2] == 'test':
      modified_sent = sentence[:int(ent['start'])] + inserttxt + ' ' + sentence[int(ent['start']):int(ent['end'])] + ' ' + inserttxt + sentence[int(ent['end']):]
      modified_sentences.append(modified_sent)
  return modified_sentences

"""Get each sentence, the model for negation recognition and the topic's entities; returns the classified entities.
"""
def get_negations(sentence, model2, tokenizer2,topic_ent):
  classifier = TextClassificationPipeline(model=model2, tokenizer=tokenizer2)
  classification = classifier(sentence)
  return classification,topic_ent

"""Get a list of extracted entities and return the ad-hoc query for search with PyTerrier
"""
def create_adhoc(list_ner):
  if list_ner:
    return ' '.join(list_ner).replace("\'"," ").translate(str.maketrans(' ', ' ', string.punctuation))
  else:
    print("Empty Entity list")
    return None

"""Get a sentence and all its entities, uses the context algorithm on the entities and identifies their class
"""
def apply_context_med_entities(sentence, topic_ent,nlp_context):
  target_matcher = nlp_context.get_pipe("medspacy_target_matcher")
  target_rules_ = []
  entities = []

  #Adds the identified entity to the medspacy_target so this entity can be lated categorized by Context
  for ent1 in topic_ent:
    target_rules_.append(TargetRule(ent1['word'],ent1['entity_group']))
    target_matcher.add(target_rules_)

  doc = nlp_context(sentence)

  #If topic_ent catches sentences that do not have an identified entity
  if topic_ent:
    for ent in doc.ents:
      # entities.append([ent1['word'], ent._.is_negated, ent._.is_uncertain, ent._.is_family, ent._.is_historical, ent._.is_hypothetical])
      entities.append([ent1['word'], ent._.is_negated, ent._.is_family, ent._.is_historical])
      break

  for ent in doc.ents:
    # entities.append([ent, ent._.is_negated, ent._.is_uncertain, ent._.is_family, ent._.is_historical, ent._.is_hypothetical])
    entities.append([ent, ent._.is_negated, ent._.is_family, ent._.is_historical])
  return entities

"""Transforms the columns of a dataframe to list
TODO: Use a more efficient way."""
def col_to_list(df):
  for column in df.columns:
    df[column] = df[column].values.tolist()
  df = df.fillna(' ')
  return df

"""Receives as input the dataframe with the lists of extracted NER per category and merges them into one
"""
def create_adhoc_from_csv(x):
  x = list(x)
  a1 = ''
  for a in x:
    a1 = a1 + ' '+ a.translate(str.maketrans(' ', ' ', string.punctuation))
  return a1

# """Expand the spacy nlp with abbreviation detector and umls expander
# """
# def create_nlp_umls(nlp_sentence):
#   nlp_sentence.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls", "max_entities_per_mention": 1, 'k':10})
#   nlp_sentence.add_pipe('abbreviation_detector')
#   linker = nlp_sentence.get_pipe("scispacy_linker")
#   return nlp_sentence,linker

# nlp_umls,linker = create_nlp_umls(nlp_sentence)

https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmp46z2r535
Finished download, copying /tmp/tmp46z2r535 to cache at /root/.scispacy/datasets/2b79923846fb52e62d686f2db846392575c8eb5b732d9d26cd3ca9378c622d40.87bd52d0f0ee055c1e455ef54ba45149d188552f07991b765da256a1b512ca0b.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmp1kiogvf9
Finished download, copying /tmp/tmp1kiogvf9 to cache at /root/.scispacy/datasets/7e8e091ec80370b87b1652f461eae9d926e543a403a69c1f0968f71157322c25.6d801a1e14867953e36258b0e19a23723ae84b0abd2a723bdd3574c3e0c873b4.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmp4m_nquj6
Finished download, copying /tmp/tmp4m_nquj6 to cache at /root/.scispacy/da

Exception ignored in: <function _xla_gc_callback at 0x7f628ce54940>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmptaf9bn1b
Finished download, copying /tmp/tmptaf9bn1b to cache at /root/.scispacy/datasets/21a1012c532c3a431d60895c509f5b4d45b0f8966c4178b892190a302b21836f.330707f4efe774134872b9f77f0e3208c1d30f50800b3b39a6b8ec21d9adf1b7.umls_semantic_type_tree.tsv


# Functions for identification of various NEs in the queries

## Identify Negated NEs using BERT and NegBert

Models:
1. NER - bert-base-uncased_clinical-ner
2. Negated NERs - bvanaken/clinical-assertion-negation-bert

The model expects input in the form of **spans/sentences with one marked entity to classify** as PRESENT(0), ABSENT(1) or POSSIBLE(2). The entity in question is identified with the special token [entity] surrounding it.

1. Input: Raw Topics, information needed to classify (present_ners,negated_ners,possible_ners), NERs to be extracted list: ['problem','treatment','test']
2. Output: one dedicated list with either the negated, possible and present problems, treatments or tests

Steps:
1. Given a query (raw topic), identify its entities using.
2. Then split it in sentences and put the [entity] token around it. (so that the BERT model can predict this entities status)
3. If a sentence contains two entities, two distinct inputs for the model are created.
4. Finally, for each query this method provides three list of entities, that contain the negated, possible and present entities.

In [6]:
def identify_ners(topic: str,clasify_information: str, extracted_NER: list, keep_noNER_sent: bool, remove_negated_NERS: bool):
  #Split the topic into sentences using the en_core_sci_sm spacy model
  sentences = split_sent(topic,nlp_sentence)
  #Initiate list of terms
  list_of_terms_present = []
  list_of_terms_absent = []
  list_of_terms_possible = []
  sentences_with_no_entities = []

  for sentence in sentences:
    #Use bert-base-uncased_clinical-ner to extract [Problem, treatment, test]
    topic_ent = get_entities(sentence)
    #Add the required tokens in the segment to identify Negated using NegBert
    ready_sentences = modify_segment_negation(str(sentence), topic_ent,extracted_NER)
    classification,topic_ent  = get_negations(ready_sentences, model2, tokenizer2, topic_ent)
    #Keep sentences that have no identified NER
    if not topic_ent:
      sentences_with_no_entities.append(str(sentence))

    #Create lists that contain identified entities - present, absent(aka negated), - possible
    for el in zip(topic_ent,classification):
      if str(el[1]['label']) == 'PRESENT':
        list_of_terms_present.append(str(el[0]['word']))
      if str(el[1]['label']) == 'ABSENT':
        list_of_terms_absent.append(str(el[0]['word']))
      if str(el[1]['label']) == 'POSSIBLE':
        list_of_terms_possible.append(str(el[0]['word']))

  final_ners_present = '|'.join(list_of_terms_present).replace('|##', '').split('|')
  final_ners_absent = '|'.join(list_of_terms_absent).replace('|##', '').split('|')
  final_ners_possible = '|'.join(list_of_terms_possible).replace('|##', '').split('|')

  #returns all extracted NERS
  if remove_negated_NERS == False:
    if keep_noNER_sent:
      return (final_ners_present+final_ners_absent+final_ners_possible+sentences_with_no_entities)
    else:
      return (final_ners_present+final_ners_absent+final_ners_possible)
  else:
    #returns requested extracted NERS

    #returns only the present problems,treatments or test
    if clasify_information == 'present_ners':
      if not final_ners_present[0]:
        print('No information.')
        return None
      else:
        #additionally returns also the sentences with no identified Entities
        if keep_noNER_sent:
          return (final_ners_present+sentences_with_no_entities)
        else:
          return final_ners_present

    #returns only the negated problems,treatments or test
    elif clasify_information == 'negated__ners':
      if not final_ners_absent[0]:
        print('No information.')
        return None
      else:
        #additionally returns also the sentences with no identified Entities
        if keep_noNER_sent:
          return (final_ners_absent+sentences_with_no_entities)
        else:
          return final_ners_absent

    #returns only the possible problems,treatments or test
    elif clasify_information == 'possible_ners':
      if not final_ners_possible[0]:
        print('No information.')
        return None
      else:
        #additionally returns also the sentences with no identified Entities
        if keep_noNER_sent:
          return (final_ners_possible+sentences_with_no_entities)
        else:
          return final_ners_possible


### Testing:
##def identify_ners(topic: str,clasify_information: str, extracted_NER: list, keep_noNER_sent: bool, remove_negated_NERS: bool):

display(queries['text'][33])

print('All identified NES + Sentences without NERS \n') #keep_noNER_sent: bool, remove_negated_NERS: bool
display(create_adhoc(identify_ners(queries['text'][33],'present_ners',['problem','treatment','test'],True,False)))
print('All not_negated NES \n')
display(create_adhoc(identify_ners(queries['text'][33],'present_ners',['problem','treatment','test'],False,True)))
print('All not_negated NES + Sentences without NERS \n')
display(create_adhoc(identify_ners(queries['text'][33],'present_ners',['problem','treatment','test'],True,True)))
print('All NES\n')
display(create_adhoc(identify_ners(queries['text'][33],'present_ners',['problem','treatment','test'],False,False)))

'\nA 17-year-old male comes to the office due to several months of right elbow pain.  The pain is worse with activity and limits his workouts and activities.  He has tried over-the-counter medications with limited relief.  Medical history is notable for eczema, and current medications include a topical hydrocortisone ointment.  He is sexually active with his girlfriend and uses condoms. He does not smoke or drink alcohol. He plays tennis most of the days of the week. The comprehensive evaluation shows pain on the lateral side of the elbow, made worse by pressure applied on the lateral epicondyle of the humerus and when making a fist with the elbow joint straightened. The patient has this pain since last year and had several courses of physical therapy.\n'

All identified NERS + Sentences without NERS 



No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


'right elbow pain the pain over  the  counter medications eczema current medications a topical hydrocortisone ointment the comprehensive evaluation pain on the lateral side of the elbow pressure this pain physical therapy    He is sexually active with his girlfriend and uses condoms He does not smoke or drink alcohol He plays tennis most of the days of the week \n'

All not_negated NERS 



'right elbow pain the pain over  the  counter medications eczema current medications a topical hydrocortisone ointment the comprehensive evaluation pain on the lateral side of the elbow pressure this pain physical therapy'

All not_negated NERS + Sentences without NERS 



'right elbow pain the pain over  the  counter medications eczema current medications a topical hydrocortisone ointment the comprehensive evaluation pain on the lateral side of the elbow pressure this pain physical therapy  He is sexually active with his girlfriend and uses condoms He does not smoke or drink alcohol He plays tennis most of the days of the week \n'

All NERS



'right elbow pain the pain over  the  counter medications eczema current medications a topical hydrocortisone ointment the comprehensive evaluation pain on the lateral side of the elbow pressure this pain physical therapy  '

## Identify Negated Entities, Family History and Temporal/historical Information using the ConText algorithm

Models:
1. NERs - NERs -- MODEL samrawal/bert-base-uncased_clinical-ner [Problem, treatment, test]
2. NER - using Med spacy
3. Med_spacy for applying ConText and categorizing the entities


1. Input: Raw Topics, information to be classified non_negated,family_history, historical_information
2. Output: One list with the NOT ent._.is_negated (so present information), ent._.is_family, ent._.is_historical

Steps:
1. Split the topic into sentences using spacy sci model
2. Get entities with the NERs Bert model
3. Annotate the entities with the ConText Algorithm

In [8]:
def apply_context(topic: str,extracted_information: list,keep_noNER_sent: bool):
  #Splits the topic into sentences using nlp spacy
  nlp_context = medspacy.load()
  sentences = split_sent(topic,nlp_sentence)
  topic_entities = []
  #Not_negated,
  list_of_terms_present = []
  #family history
  list_of_terms_family = []
  #historical
  list_of_terms_historical = []
  #Not_negated, not family, not historical
  list_of_terms_000 = []
  #Not_negated,family, not historical
  list_of_terms_010 = []
  #Not_negated, not family history, historical
  list_of_terms_001 = []
  #Not_negated, family history, historical
  list_of_terms_011 = []

  sentences_with_no_entities = []

  for sentence in sentences:
    #Using bert-base-uncased_clinical-ner identify [Problem, treatment, test]
    topic_ent = get_entities(sentence)
    #For each sentence, the entity is categorized as negated, family or historical
    ready_sentences = apply_context_med_entities(str(sentence), topic_ent,nlp_context)
    topic_entities.append(ready_sentences)

    if not topic_ent:
      sentences_with_no_entities.append(str(sentence))

  #Create the query based on the entitie type:
  for sent_entities in topic_entities:
    for entity in sent_entities:
      #Creates list with not_negated information
      if entity[1] == False:
        list_of_terms_present.append(str(entity[0]))
      #Creates list with all family related information
      if entity[2] == True:
        list_of_terms_family.append(str(entity[0]))
      #Creates list with all historical information
      if entity[3] == True:
        list_of_terms_historical.append(str(entity[0]))
      #Creates list with not_negated, not family history and not historical information [combinations]
      if entity[1] == False and entity[2] == False and entity[3] == False:
        list_of_terms_000.append(str(entity[0]))
      #Creates list with not_negated, family history and not historical information [combinations]
      if entity[1] == False and entity[2] == True and entity[3] == False:
        list_of_terms_010.append(str(entity[0]))
      #Creates list with not_negated, not family history and historical information [combinations]
      if entity[1] == False and entity[2] == False and entity[3] == True:
        list_of_terms_001.append(str(entity[0]))
      #Creates list with not_negated, family history and historical information [combinations]
      if entity[1] == False and entity[2] == True and entity[3] == True:
        list_of_terms_011.append(str(entity[0]))

  final_ners_present = '|'.join(list_of_terms_present).replace('|##', '').split('|')
  final_ners_fam = '|'.join(list_of_terms_family).replace('|##', '').split('|')
  final_ners_hist = '|'.join(list_of_terms_historical).replace('|##', '').split('|')
  final_ners_000 = '|'.join(list_of_terms_000).replace('|##', '').split('|')
  final_ners_010 = '|'.join(list_of_terms_010).replace('|##', '').split('|')
  final_ners_001 = '|'.join(list_of_terms_001).replace('|##', '').split('|')
  final_ners_011 = '|'.join(list_of_terms_011).replace('|##', '').split('|')

  #Returns all not_negated NERs - Problems, treatments, tests
  if extracted_information[0] == 'not_negated' and extracted_information[1] == 'None'and extracted_information[2] == 'None':
    if not final_ners_present[0]:
      print('All information is negated.')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_present+sentences_with_no_entities)
      else:
          return final_ners_present
  #Returns all family_related NERs - Problems, treatments, tests
  if extracted_information[0] == 'None' and extracted_information[1] == 'family_history'and extracted_information[2] == 'None':
    if not final_ners_fam[0]:
      print('No family history.')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_fam+sentences_with_no_entities)
      else:
          return final_ners_fam

  #Returns all historical NERs - Problems, treatments, tests
  if extracted_information[0] == 'None' and extracted_information[1] == 'None'and extracted_information[2] == 'historical_information':
    if not final_ners_hist[0]:
      print('No past medical history.')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_hist+sentences_with_no_entities)
      else:
        return final_ners_hist

  #Returns all not_negated, not_family,not_historical NERs - Problems, treatments, tests
  if extracted_information[0] == 'not_negated' and extracted_information[1] == 'not_family_history'and extracted_information[2] == 'not_historical_information':
    if not final_ners_000[0]:
      print('Empty query, not identified NERS')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_000+sentences_with_no_entities)
      else:
          return final_ners_000
  #Returns all not_negated, family NERs - Problems, treatments, tests
  if extracted_information[0] == 'not_negated' and extracted_information[1] == 'family_history' and extracted_information[2] == 'not_historical_information':
    if not final_ners_010:
      print('Empty query, not identified NERS')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_010+sentences_with_no_entities)
      else:
          return final_ners_010
  #Returns all not_negated, not_family, historical NERs - Problems, treatments, tests
  if extracted_information[0] == 'not_negated' and extracted_information[1] == 'not_family_history'and extracted_information[2] == 'historical_information':
    if not final_ners_001:
      print('Empty query, not identified NERS')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_001+sentences_with_no_entities)
      else:
          return final_ners_001

  #Returns all not_negated, family, historical NERs - Problems, treatments, tests
  if (extracted_information[0] == 'not_negated' and extracted_information[1] == 'family_history' and extracted_information[2] == 'historical_information'):
    if not final_ners_011:
      print('Empty query, not identified NERS')
      return None
    else:
      if keep_noNER_sent:
          return (final_ners_011+sentences_with_no_entities)
      else:
          return final_ners_011


### Testing:
display(queries['text'][33])
print(create_adhoc(apply_context(queries['text'][33], ['not_negated','not_family_history','not_historical_information'],True)))
print(create_adhoc(apply_context(queries['text'][33], ['not_negated','not_family_history','not_historical_information'],False)))

'\nA 17-year-old male comes to the office due to several months of right elbow pain.  The pain is worse with activity and limits his workouts and activities.  He has tried over-the-counter medications with limited relief.  Medical history is notable for eczema, and current medications include a topical hydrocortisone ointment.  He is sexually active with his girlfriend and uses condoms. He does not smoke or drink alcohol. He plays tennis most of the days of the week. The comprehensive evaluation shows pain on the lateral side of the elbow, made worse by pressure applied on the lateral epicondyle of the humerus and when making a fist with the elbow joint straightened. The patient has this pain since last year and had several courses of physical therapy.\n'

right elbow pain right elbow pain the pain The pain over  the  counter medications overthecounter medications pressure The comprehensive evaluation pain on the lateral side of the elbow pressure physical therapy this pain physical therapy  He is sexually active with his girlfriend and uses condoms He does not smoke or drink alcohol He plays tennis most of the days of the week 

right elbow pain right elbow pain the pain The pain over  the  counter medications overthecounter medications pressure The comprehensive evaluation pain on the lateral side of the elbow pressure physical therapy this pain physical therapy


# Creation of the various query representations



In [10]:
experiment = 'Bert_All_not_negated_NERs_keep_Sentences'
extract = ['problem','treatment','test']

#Keep sentences that do not contain any medical NER (keep_noNER_sent=True)
#Remove_negated_NERS (remove_negated_NERS=True)
queries['processed'] = queries['text'].apply(lambda x: (create_adhoc(identify_ners(x,'present_ners',extract,keep_noNER_sent=True,remove_negated_NERS=True))))
display(queries)

,query_id,text,processed
0,1,\nA 19-year-old male came to clinic with some ...,sexual concern physical examination some pubic...
1,2,\nA 32-year-old woman comes to the hospital wi...,vaginal spotting appendectomy several complica...
2,3,\nA 51-year-old man comes to the office compla...,fatigue some sexual problems lack of libido bl...
3,4,\nA 66-year-old woman comes to the office due ...,joint pain in the hands periodic morning stiff...
4,5,\nA 23-year-old man comes to the emergency dep...,syncope dizzy light headedness vital signs ph...
5,6,\nA 61-year-old man comes to the clinic due to...,nonproductive cough progressive dyspnea hypert...
6,7,\nA 3-year-old girl is brought to the clinic b...,assessment her short stature physical examinat...
7,8,\nA 7-month-old boy is brought to emergency by...,irritability inability to defecate constipatio...
8,9,\nA 67-year-old woman comes to the clinic due ...,choking dysphagia cough her other medical prob...
9,10,\nA 19-year-old girl comes to the clinic due t...,a left wrist mass swelling on the top of her w...
